In [1]:
import cymetric as cym
from cymetric import timeseries
import matplotlib as plt 
import pandas as pd
import numpy as np
import sys 
sys.path.insert(0, '../../../scripts/')
import output as oup

# This is when you already have the results 

In [2]:
df = pd.read_csv('fs-df.csv',index_col='FS')
df

,Final HLW,Final Depleted U,Total uranium ore,Total idle capacity,Last date idle capacity,Duration of transition
FS,,,,,,
0,1.315306e+07,7.988186e+08,1.437000e+11,135.083333,962,2
5,1.305699e+07,7.988186e+08,1.437000e+11,135.083333,962,2
10,1.305190e+07,7.988186e+08,1.437000e+11,135.083333,962,2
15,1.295955e+07,7.988186e+08,1.437000e+11,135.083333,962,2
20,1.300212e+07,7.988186e+08,1.437000e+11,135.083333,962,2


# The rest of the code below is to generate the above results

In [3]:
starter_string = 'FS'
scenario_nums = ['0','5','10','15','20']

In [ ]:
df = oup.initialize_df(scenario_index=starter_string,
                                 scenarios_nums=scenario_nums)

In [ ]:
df['Final HLW'] = 0
df['Final Depleted U'] = 0 
df['Total uranium ore'] = 0
df['Total idle capacity'] = 0
df['Last date idle capacity'] = 0
df['Duration of transition'] = 0

In [ ]:
output_start = '../cyclus-files/oat/fleet-share/fs'
ev_dict = {}
for x in range(len(scenario_nums)): 
    output_file = output_start + scenario_nums[x]+'.sqlite'
    ev_dict[scenario_nums[x]] = cym.Evaluator(db=cym.dbopen(output_file),write=True)

In [ ]:
for x in range(5):#(len(scenario_nums)): 
    df.loc[scenario_nums[x],'Final HLW'] = cym.timeseries.transactions(ev_dict[scenario_nums[x]],commodities=['lwrreprocessingwaste','moxreprocessingwaste','frreprocessingwaste'])['Mass'].cumsum().iloc[-1]
    df.loc[scenario_nums[x],'Final Depleted U'] = cym.timeseries.transactions(ev_dict[scenario_nums[x]],commodities=['enrichmentwaste'])['Mass'].cumsum().max()
    df.loc[scenario_nums[x],'Total uranium ore'] = cym.timeseries.transactions(ev_dict[scenario_nums[x]],commodities=['sourceout'])['Mass'].cumsum().max()
    df.loc[scenario_nums[x],'Total idle capacity'], df.loc[scenario_nums[x],'Last date idle capacity'], udict = oup.idlecap(ev_dict[scenario_nums[x]],'(60000+250*t/12)/1000')
    df.loc[scenario_nums[x],'Duration of transition'] = df.loc[scenario_nums[x],'Last date idle capacity'] - 960
    

In [ ]:
df.to_csv('fs-df.csv')

In [5]:
df_fs_sa = oup.sensitivity(base_case=15,init_df=df)
df_fs_sa

,Final HLW,Final Depleted U,Total uranium ore,Total idle capacity,Last date idle capacity,Duration of transition
FS,,,,,,
0,1.493165,0.0,0.0,0.0,0.0,0.0
5,0.751836,0.0,0.0,0.0,0.0,0.0
10,0.712542,0.0,0.0,0.0,0.0,0.0
15,0.000000,0.0,0.0,0.0,0.0,0.0
20,0.328459,0.0,0.0,0.0,0.0,0.0


In [6]:
df_fs_sa.to_csv('fs-df-sa.csv')